In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")
oto = pd.read_json("scout_car.json",orient="recordings", lines=True)
pd.options.display.max_columns = 600
pd.options.display.max_rows = 20

In [2]:
#Functions :
#convert lists values to new columns with this no need to change column type to string or NAN to [] values normally requires before applying below.
def list_to_columns(df,col): 
    dftemp = pd.DataFrame([pd.Series(x) for x in df[col]]) # that is better and quicker. no need to run NAN_to_list.?
    column_names= []
    max_element = len(max(df.loc[df[col].apply(lambda x : isinstance(x, list))][col], key=len))
    #max_element = len(max(df.loc[df[col].notna()][col], key=len))
    for i in range(max_element):
        column_names.append(col + str(i+1))
    dftemp.columns = column_names
    df2 = df.drop(col,axis=1)
    if max_element == 1:
        dftemp.rename(columns={col+'1': col},inplace = True)
    df_added = pd.concat([df2, dftemp], axis=1, sort=False)
    return df_added

# check a column if it contains list of values. returns 2 list that we need to define
def check_is_list(df):
    list_of_columns = []
    non_list_of_columns = []
    for i in df.columns:
        counter_list=0
        counter_non_list=0
        for j in df[i].values:
            if isinstance(j, list):
                counter_list+=1
            else:
                counter_non_list+=1
        if counter_list > 0:
            list_of_columns.append(i)
        else :
            non_list_of_columns.append(i)
    return list_of_columns,non_list_of_columns
#changes a phrase in a list to another value
def del_str_phrase(phrase,liste):
    while(phrase in liste) : 
        liste.remove(phrase)
        
#get the df colon without outliers specified
def get_df_wo_outlier(df,col,range=1.5):
    Q1 = df[col].quantile(q=0.25)
    Q3 = df[col].quantile(q=0.75)
    IQR = Q3-Q1
    return df[col][~((df[col] <  Q1-(range*IQR)) |(df[col] > Q3+(range*IQR))) ]

#to have a look at data, which columns contain what and how many.
def get_all_df(df):
    for i in df.columns:
        print(df[i].value_counts(dropna = False).rename_axis(i +' unique_values').reset_index(name=i +' counts'))

#quick look on a column        
def get_col_details(df,col):
    print("VALUE COUNTS")
    print(df[col].value_counts(dropna=False))
    print("-----------------------\n","UNIQUE VALUES")
    print(df[col].unique())
    print("-----------------------\n","NULL SUM")
    print(df[col].isnull().sum())
    print("-----------------------\n","NULL Percentage")
    print(df[col].isnull().sum()*100 /df[col].index.stop )
    print("-----------------------\n","COLUMN with NONE-NULL-NAN")
    print(df.query(f'{col} != {col}')[col])
    
#updates values NAN to empty list : []
def Nan_to_list(df,col):
    Nan_rows = df[col].isnull()
    df.loc[Nan_rows,col] = df.loc[Nan_rows,col].apply(lambda x : [])
    
def outlier_zscore(df, col, min_z=1, max_z = 5, step = 0.1, print_list = False):
    z_scores = zscore(df[col].dropna())
    threshold_list = []
    for threshold in np.arange(min_z, max_z, step):
        threshold_list.append((threshold, len(np.where(z_scores > threshold)[0])))
        df_outlier = pd.DataFrame(threshold_list, columns = ['threshold', 'outlier_count'])
        df_outlier['pct'] = (df_outlier.outlier_count - df_outlier.outlier_count.shift(-1))/df_outlier.outlier_count*100
    plt.plot(df_outlier.threshold, df_outlier.outlier_count)
    best_treshold = round(df_outlier.iloc[df_outlier.pct.argmax(), 0],2)
    outlier_limit = int(df[col].dropna().mean() + (df[col].dropna().std()) * df_outlier.iloc[df_outlier.pct.argmax(), 0])
    percentile_threshold = stats.percentileofscore(df[col].dropna(), outlier_limit)
    plt.vlines(best_treshold, 0, df_outlier.outlier_count.max(), 
               colors="r", ls = ":"
              )
    plt.annotate("Zscore : {}\nValue : {}\nPercentile : {}".format(best_treshold, outlier_limit, 
                                                                   (np.round(percentile_threshold, 3), 
                                                                    np.round(100-percentile_threshold, 3))), 
                 (best_treshold, df_outlier.outlier_count.max()/2))
    #plt.show()
    if print_list:
        print(df_outlier)
    return (plt, df_outlier, best_treshold, outlier_limit, percentile)

def outlier_inspect(df, col, min_z=1, max_z = 5, step = 0.5, max_hist = None, bins = 50):
    fig = plt.figure(figsize=(20, 6))
    fig.suptitle(col, fontsize=16)
    plt.subplot(1,3,1)
    if max_hist == None:
        sns.distplot(df[col], kde=False, bins = 50)
    else :
        sns.distplot(df[df[col]<=max_hist][col], kde=False, bins = 50)
    plt.subplot(1,3,2)
    sns.boxplot(df[col])
    plt.subplot(1,3,3)
    z_score_inspect = outlier_zscore(df, col, min_z=min_z, max_z = max_z, step = step)
    plt.show()

In [3]:
#!pip install pandas-profiling
#import pandas_profiling


In [4]:
#pandas_profiling.ProfileReport(df)

In [5]:
#oto.rename(columns=lambda x: x.replace("\n",""),inplace=True)

In [6]:
list(oto.columns)

['url',
 'make_model',
 'short_description',
 'body_type',
 'price',
 'vat',
 'km',
 'registration',
 'prev_owner',
 'kW',
 'hp',
 'Type',
 'Previous Owners',
 'Next Inspection',
 'Inspection new',
 'Warranty',
 'Full Service',
 'Non-smoking Vehicle',
 'null',
 'Make',
 'Model',
 'Offer Number',
 'First Registration',
 'Body Color',
 'Paint Type',
 'Body Color Original',
 'Upholstery',
 'Body',
 'Nr. of Doors',
 'Nr. of Seats',
 'Model Code',
 'Gearing Type',
 'Displacement',
 'Cylinders',
 'Weight',
 'Drive chain',
 'Fuel',
 'Consumption',
 'CO2 Emission',
 'Emission Class',
 '\nComfort & Convenience\n',
 '\nEntertainment & Media\n',
 '\nExtras\n',
 '\nSafety & Security\n',
 'description',
 'Emission Label',
 'Gears',
 'Country version',
 'Electricity consumption',
 'Last Service Date',
 'Other Fuel Types',
 'Availability',
 'Last Timing Belt Service Date',
 'Available from']

# 18sept

In [7]:
oto["body_type"].value_counts(dropna = False)


Sedans           7903
Station wagon    3553
Compact          3153
Van               783
Other             290
Transporter        88
NaN                60
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64

In [8]:
oto[oto["make_model"] == "Audi A3"]["body_type"].mode()[0]

'Sedans'

In [9]:
grp_inx = list(oto[oto['make_model'] == 'Audi A3']['body_type'].index)
grp_inx

[2615,
 2616,
 2617,
 2618,
 2619,
 2620,
 2621,
 2622,
 2623,
 2624,
 2625,
 2626,
 2627,
 2628,
 2629,
 2630,
 2631,
 2632,
 2633,
 2634,
 2635,
 2636,
 2637,
 2638,
 2639,
 2640,
 2641,
 2642,
 2643,
 2644,
 2645,
 2646,
 2647,
 2648,
 2649,
 2650,
 2651,
 2652,
 2653,
 2654,
 2655,
 2656,
 2657,
 2658,
 2659,
 2660,
 2661,
 2662,
 2663,
 2664,
 2665,
 2666,
 2667,
 2668,
 2669,
 2670,
 2671,
 2672,
 2673,
 2674,
 2675,
 2676,
 2677,
 2678,
 2679,
 2680,
 2681,
 2682,
 2683,
 2684,
 2685,
 2686,
 2687,
 2688,
 2689,
 2690,
 2691,
 2692,
 2693,
 2694,
 2695,
 2696,
 2697,
 2698,
 2699,
 2700,
 2701,
 2702,
 2703,
 2704,
 2705,
 2706,
 2707,
 2708,
 2709,
 2710,
 2711,
 2712,
 2713,
 2714,
 2715,
 2716,
 2717,
 2718,
 2719,
 2720,
 2721,
 2722,
 2723,
 2724,
 2725,
 2726,
 2727,
 2728,
 2729,
 2730,
 2731,
 2732,
 2733,
 2734,
 2735,
 2736,
 2737,
 2738,
 2739,
 2740,
 2741,
 2742,
 2743,
 2744,
 2745,
 2746,
 2747,
 2748,
 2749,
 2750,
 2751,
 2752,
 2753,
 2754,
 2755,
 2756,
 2757,

In [10]:
doldur = oto[oto["make_model"] == "Audi A3"]["body_type"].mode()[0]

In [11]:
oto["body_type"].iloc[grp_inx] = oto["body_type"].iloc[grp_inx].fillna(doldur)

In [12]:
oto["body_type"].value_counts(dropna = False)



Sedans           7910
Station wagon    3553
Compact          3153
Van               783
Other             290
Transporter        88
Off-Road           56
NaN                53
Coupe              25
Convertible         8
Name: body_type, dtype: int64

In [13]:
def fill_most(df, group_col, col_name):
    for group in list(df[group_col].unique()):
        grp_inx = list(df[df[group_col]==group][col_name].index)
        df[col_name].iloc[grp_inx] = df[col_name].iloc[grp_inx].fillna(df[df[group_col]==group][col_name].mode()[0])

In [14]:
fill_most(oto,"make_model","body_type")

In [15]:
#fill_most(df,"make_model","Extras")

In [16]:
#fill_most(df,"make_model","Comfort & Convenience")

In [17]:
oto["body_type"].value_counts(dropna = False)


Sedans           7925
Station wagon    3563
Compact          3155
Van               809
Other             290
Transporter        88
Off-Road           56
Coupe              25
Convertible         8
Name: body_type, dtype: int64

# next inspection

In [18]:
"".join([i for i in oto["Next Inspection"][0]])

'\n06/2021\n\n99 g CO2/km (comb)\n'

In [19]:
["".join(item).strip() if type(item)==list else item for item in oto["Next Inspection" ] ]

['06/2021\n\n99 g CO2/km (comb)',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 '02/2020\n\n97 g CO2/km (comb)',
 nan,
 nan,
 nan,
 '09/2019\n\n112 g CO2/km (comb)',
 nan,
 '10/2019\n\n99 g CO2/km (comb)',
 '04/2020\n\n97 g CO2/km (comb)',
 nan,
 nan,
 '\n07/2020\n',
 nan,
 nan,
 nan,
 '12/2019\n\n112 g CO2/km (comb)',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 '05/2019\n\n95 g CO2/km (comb)',
 nan,
 nan,
 nan,
 nan,
 nan,
 '07/2019\n\n97 g CO2/km (comb)',
 '10/2019\n\n97 g CO2/km (comb)',
 nan,
 nan,
 nan,
 nan,
 nan,
 '02/2021\n\n94 g CO2/km (comb)',
 '06/2019\n\n99 g CO2/km (comb)',
 nan,
 nan,
 nan,
 nan,
 nan,
 '06/2019\n\n97 g CO2/km (comb)',
 nan,
 nan,
 nan,
 '05/2020\n\n102 g CO2/km (comb)',
 '11/2019\n\n97 g CO2/km (comb)',
 '06/2021\n\n102 g CO2/km (comb)',
 '12/2019\n\n99 g CO2/km (comb)',
 nan,
 nan,
 nan,
 nan,
 '06/2021\n\n102 g CO2/km (comb)',
 nan,
 '\n11/2020\n',
 nan,
 nan,
 nan,
 '09/2021\n\n104 g CO2/km (comb)',
 nan,
 '03/2020\n\n117 g CO2/km (comb)',
 nan,
 na

In [20]:
#now use extract etc.

# # 19sept

In [21]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_info_columns', 500)
pd.set_option('display.max_info_rows', 2000)
pd.set_option('display.expand_frame_repr', True)
pd.set_option('display.width', 2000)

In [22]:
oto.Consumption

0                  [[3.8 l/100 km (comb)], [4.3 l/100 km (city)], [3.5 l/100 km (country)]]
1                  [[5.6 l/100 km (comb)], [7.1 l/100 km (city)], [4.7 l/100 km (country)]]
2                  [[3.8 l/100 km (comb)], [4.4 l/100 km (city)], [3.4 l/100 km (country)]]
3                  [[3.8 l/100 km (comb)], [4.3 l/100 km (city)], [3.5 l/100 km (country)]]
4                  [[4.1 l/100 km (comb)], [4.6 l/100 km (city)], [3.8 l/100 km (country)]]
                                                ...                                        
15914              [[5.3 l/100 km (comb)], [6.2 l/100 km (city)], [4.7 l/100 km (country)]]
15915    [\n, 7.4 l/100 km (comb), \n, 9.2 l/100 km (city), \n, 6.3 l/100 km (country), \n]
15916              [[5.3 l/100 km (comb)], [6.2 l/100 km (city)], [4.7 l/100 km (country)]]
15917              [[5.3 l/100 km (comb)], [6.2 l/100 km (city)], [4.7 l/100 km (country)]]
15918              [[6.8 l/100 km (comb)], [8.7 l/100 km (city)], [5.7 l/100 km 

In [23]:
oto.Consumption.str[0].str[0].str.split()

0        [3.8, l/100, km, (comb)]
1        [5.6, l/100, km, (comb)]
2        [3.8, l/100, km, (comb)]
3        [3.8, l/100, km, (comb)]
4        [4.1, l/100, km, (comb)]
                   ...           
15914    [5.3, l/100, km, (comb)]
15915                          []
15916    [5.3, l/100, km, (comb)]
15917    [5.3, l/100, km, (comb)]
15918    [6.8, l/100, km, (comb)]
Name: Consumption, Length: 15919, dtype: object

In [24]:
oto.Consumption.str[1].str[0].str.split()

0        [4.3, l/100, km, (city)]
1        [7.1, l/100, km, (city)]
2        [4.4, l/100, km, (city)]
3        [4.3, l/100, km, (city)]
4        [4.6, l/100, km, (city)]
                   ...           
15914    [6.2, l/100, km, (city)]
15915                         [7]
15916    [6.2, l/100, km, (city)]
15917    [6.2, l/100, km, (city)]
15918    [8.7, l/100, km, (city)]
Name: Consumption, Length: 15919, dtype: object

In [25]:
# a = oto['Comfort & Convenience'].explode().unique()
# max_list = sorted([i for i in a if not pd.isnull(list(a)[list(a).index(i)])])
# max_list

In [27]:
oto["Model Code"].str[0].str.strip()


0        0588/BDF
1        0588/BCY
2             NaN
3             NaN
4        0588/BDF
           ...   
15914         NaN
15915    0000/000
15916         NaN
15917         NaN
15918    3333/BHJ
Name: Model Code, Length: 15919, dtype: object

In [28]:
oto["Body Color"].str[1].value_counts(dropna = False).head(10)

Black     3745
Grey      3505
White     3406
Silver    1647
Blue      1431
Red        957
NaN        597
Brown      289
Green      154
Beige      108
Name: Body Color, dtype: int64

In [29]:
oto["Body Color"].fillna(method="bfill",inplace=True)

In [30]:
oto["Body Color"].str[1].value_counts(dropna = False).head(10)

Black     3888
Grey      3638
White     3540
Silver    1687
Blue      1524
Red        989
Brown      299
Green      163
Beige      108
Yellow      53
Name: Body Color, dtype: int64

In [31]:

oto["CO2 Emission"].str[1].value_counts(dropna = False).head(10)

NaN    15291
[]       628
Name: CO2 Emission, dtype: int64

In [32]:
get_col_details(oto,"CO2 Emission")

VALUE COUNTS
NaN                            1808
[\n120 g CO2/km (comb)\n]       740
[[], [], []]                    628
[\n99 g CO2/km (comb)\n]        545
[\n97 g CO2/km (comb)\n]        537
                               ... 
[\n193 g CO2/km (comb)\n]         1
[\n171 g CO2/km (comb)\n]         1
[\n990 g CO2/km (comb)\n]         1
[\n1,060 g CO2/km (comb)\n]       1
[\n183 g CO2/km (comb)\n]         1
Name: CO2 Emission, Length: 124, dtype: int64
-----------------------
 UNIQUE VALUES


TypeError: unhashable type: 'list'

In [33]:
oto['CO2 Emission'].str[0].str.strip('\n').value_counts(dropna = False).sample(55)

105 g CO2/km (comb)    202
170 g CO2/km (comb)     10
125 g CO2/km (comb)    132
118 g CO2/km (comb)    270
51 g CO2/km (comb)       1
                      ... 
96 g CO2/km (comb)      22
109 g CO2/km (comb)    234
127 g CO2/km (comb)    257
111 g CO2/km (comb)    237
253 g CO2/km (comb)      1
Name: CO2 Emission, Length: 55, dtype: int64

In [34]:
oto['CO2 Emission'].str[0].str.strip('\n').value_counts(dropna = False).sample(20)



127 g CO2/km (comb)    257
152 g CO2/km (comb)     28
158 g CO2/km (comb)     17
187 g CO2/km (comb)     26
183 g CO2/km (comb)      1
154 g CO2/km (comb)     40
159 g CO2/km (comb)      7
115 g CO2/km (comb)     86
99 g CO2/km (comb)     545
104 g CO2/km (comb)    501
84 g CO2/km (comb)       2
112 g CO2/km (comb)    171
153 g CO2/km (comb)    113
118 g CO2/km (comb)    270
1 g CO2/km (comb)        1
0 g CO2/km (comb)        2
45 g CO2/km (comb)       1
95 g CO2/km (comb)     161
144 g CO2/km (comb)     40
113 g CO2/km (comb)    235
Name: CO2 Emission, dtype: int64

In [35]:
oto['CO2 Emission'].str[0].str.strip('\n').str.replace("g CO2/km","")

0         99  (comb)
1        129  (comb)
2         99  (comb)
3         99  (comb)
4        109  (comb)
            ...     
15914    139  (comb)
15915    168  (comb)
15916    139  (comb)
15917    139  (comb)
15918    153  (comb)
Name: CO2 Emission, Length: 15919, dtype: object

In [36]:
oto["CO2 Emission"].str[0].str.strip("\n").str.replace("[^0-9]","")

0         992
1        1292
2         992
3         992
4        1092
         ... 
15914    1392
15915    1682
15916    1392
15917    1392
15918    1532
Name: CO2 Emission, Length: 15919, dtype: object

In [37]:
oto["CO2 Emission"].str.extract("(\d{1,5})")

,0
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
15914,NaN
15915,NaN
15916,NaN
15917,NaN


In [38]:
get_col_details(oto,"Displacement")

VALUE COUNTS
[\n1,598 cc\n]     4761
[\n999 cc\n]       2438
[\n1,398 cc\n]     1314
[\n1,399 cc\n]      749
[\n1,229 cc\n]      677
                   ... 
[\n1,800 cc\n]        1
[\n140 cc\n]          1
[\n15,898 cc\n]       1
[\n1,686 cc\n]        1
[\n1,368 cc\n]        1
Name: Displacement, Length: 78, dtype: int64
-----------------------
 UNIQUE VALUES


TypeError: unhashable type: 'list'

In [39]:
oto["Displacement"].str[0].str.strip().str.replace(",","").str.replace(" cc","").value_counts().unique()

array([4761, 2438, 1314,  749,  677,  670,  595,  559,  467,  353,  351,
        320,  301,  288,  212,  210,  196,  130,  110,  103,  102,   90,
         72,   50,   46,   40,   36,   25,   20,   18,   11,    6,    5,
          4,    3,    2,    1])

In [40]:
oto.replace({"Displacement" : {"," : "", " " : "", "cc" : ""}}, regex = True,inplace = True)

In [41]:
oto["Displacement"]

0        [\n1,422 cc\n]
1        [\n1,798 cc\n]
2        [\n1,598 cc\n]
3        [\n1,422 cc\n]
4        [\n1,422 cc\n]
              ...      
15914    [\n1,997 cc\n]
15915    [\n1,798 cc\n]
15916    [\n1,997 cc\n]
15917    [\n1,997 cc\n]
15918    [\n1,798 cc\n]
Name: Displacement, Length: 15919, dtype: object

In [42]:
oto["Fuel"].str[1].apply(lambda x : x.replace(" (Particulate Filter)","")).apply(lambda x : "Diesel" if "Diesel" in x else\
("Gasoline" if ( ("Super" in x) or ("Benzine" in x)or ("Gasoline (Particulate Filter)" in x)  ) else\
("gas" if ( ("CNG" in x) or ("LPG" in x)or ("gas" in x)  ) else x))).value_counts(dropna = False)

Gasoline             8545
Diesel               7299
gas                    64
Others                  6
Electric/Gasoline       4
Electric                1
Name: Fuel, dtype: int64

In [43]:
oto.replace

<bound method DataFrame.replace of                                                                                                                                                            url      make_model                                   short_description body_type  price               vat         km registration         prev_owner  kW      hp                                                       Type                                                                            Previous Owners                         Next Inspection                      Inspection new                          Warranty                      Full Service      Non-smoking Vehicle null         Make             Model          Offer Number First Registration       Body Color      Paint Type           Body Color Original                 Upholstery              Body Nr. of Doors Nr. of Seats      Model Code         Gearing Type    Displacement Cylinders          Weight  Drive chain                           

In [44]:
fuel_new = oto["Fuel"].str[1]

In [45]:
fuel_new

0                          Diesel (Particulate Filter)
1                                             Gasoline
2                          Diesel (Particulate Filter)
3                          Diesel (Particulate Filter)
4                          Diesel (Particulate Filter)
                             ...                      
15914                      Diesel (Particulate Filter)
15915    Super 95 / Super Plus 98 (Particulate Filter)
15916                                           Diesel
15917                                           Diesel
15918                                         Super 95
Name: Fuel, Length: 15919, dtype: object

In [46]:
oto.loc[e]

NameError: name 'e' is not defined

In [47]:
lpg_bool = fuel_new.str.contains("lpg|cng|bio|domestic",case = False, regex = True)

In [48]:
diesel_bool = fuel_new.str.contains("diesel",case = False, regex = True)

In [49]:
lpg_bool

0        False
1        False
2        False
3        False
4        False
         ...  
15914    False
15915    False
15916    False
15917    False
15918    False
Name: Fuel, Length: 15919, dtype: bool

In [50]:
diesel_bool

0         True
1        False
2         True
3         True
4         True
         ...  
15914     True
15915    False
15916     True
15917     True
15918    False
Name: Fuel, Length: 15919, dtype: bool

In [51]:
oto[diesel_bool]["Fuel"]

0        [\n, Diesel (Particulate Filter), \n]
2        [\n, Diesel (Particulate Filter), \n]
3        [\n, Diesel (Particulate Filter), \n]
4        [\n, Diesel (Particulate Filter), \n]
5        [\n, Diesel (Particulate Filter), \n]
                         ...                  
15911    [\n, Diesel (Particulate Filter), \n]
15912    [\n, Diesel (Particulate Filter), \n]
15914    [\n, Diesel (Particulate Filter), \n]
15916                         [\n, Diesel, \n]
15917                         [\n, Diesel, \n]
Name: Fuel, Length: 7300, dtype: object

In [52]:
oto.loc[diesel_bool,"fuel_new"] = "Diesel"

In [53]:
oto.loc[lpg_bool,"fuel_new"] = "LPG/CNG"

In [54]:
oto.fuel_new.value_counts(dropna=False)

NaN        8555
Diesel     7299
LPG/CNG      65
Name: fuel_new, dtype: int64

In [55]:
#"Super"|"Benzine"|"Gasoline (Particulate Filter)"
benzine_bool = fuel_new.str.contains("Super|Benzine|Gasoline (Particulate Filter)",case = False, regex = True)

In [56]:
oto.loc[benzine_bool,"fuel_new"] = "benzine"

In [57]:
oto.fuel_new.value_counts(dropna=False)

Diesel     7299
benzine    5293
NaN        3263
LPG/CNG      64
Name: fuel_new, dtype: int64

In [58]:
BENZLIST = list(oto.fuel_new.loc[lambda x : x !="Diesel"][lambda x : x !="LPG/CNG"].index)

In [59]:
oto.fuel_new.iloc[BENZLIST] = "Benzine"

In [60]:
oto.fuel_new.value_counts(dropna = False)

Benzine    8556
Diesel     7299
LPG/CNG      64
Name: fuel_new, dtype: int64

In [61]:
oto.loc[(~diesel_bool) & (~lpg_bool), "fuel_new"] = "gasoline" 

In [62]:
#

In [63]:
oto.Warranty

0                 [\n, \n, \n4 (Green)\n]
1                                     NaN
2        [\n, \n, \n99 g CO2/km (comb)\n]
3                                     NaN
4                    [\n, \n, \nEuro 6\n]
                       ...               
15914                       \n24 months\n
15915                [\n, \n, \nEuro 6\n]
15916             [\n, \n, \n4 (Green)\n]
15917                                  \n
15918                                 NaN
Name: Warranty, Length: 15919, dtype: object

In [ ]:
oto

In [68]:
oto.columns

Index(['url', 'make_model', 'short_description', 'body_type', 'price', 'vat', 'km', 'registration', 'prev_owner', 'kW', 'hp', 'Type', 'Previous Owners', 'Next Inspection', 'Inspection new', 'Warranty', 'Full Service', 'Non-smoking Vehicle', 'null', 'Make', 'Model', 'Offer Number', 'First Registration', 'Body Color', 'Paint Type', 'Body Color Original', 'Upholstery', 'Body', 'Nr. of Doors', 'Nr. of Seats', 'Model Code', 'Gearing Type', 'Displacement', 'Cylinders', 'Weight', 'Drive chain', 'Fuel', 'Consumption', 'CO2 Emission', 'Emission Class', '\nComfort & Convenience\n', '\nEntertainment & Media\n', '\nExtras\n', '\nSafety & Security\n', 'description', 'Emission Label', 'Gears', 'Country version', 'Electricity consumption', 'Last Service Date', 'Other Fuel Types', 'Availability', 'Last Timing Belt Service Date', 'Available from', 'fuel_new'], dtype='object')

In [81]:
oto[oto["Gears"]=="4"]

,url,make_model,short_description,body_type,price,vat,km,registration,prev_owner,kW,hp,Type,Previous Owners,Next Inspection,Inspection new,Warranty,Full Service,Non-smoking Vehicle,null,Make,Model,Offer Number,First Registration,Body Color,Paint Type,Body Color Original,Upholstery,Body,Nr. of Doors,Nr. of Seats,Model Code,Gearing Type,Displacement,Cylinders,Weight,Drive chain,Fuel,Consumption,CO2 Emission,Emission Class,\nComfort & Convenience\n,\nEntertainment & Media\n,\nExtras\n,\nSafety & Security\n,description,Emission Label,Gears,Country version,Electricity consumption,Last Service Date,Other Fuel Types,Availability,Last Timing Belt Service Date,Available from,fuel_new


In [70]:
car = pd.read_json("scout_car.json",orient="recordings", lines=True)


In [76]:
car.iloc[[15822,15823]].describe

<bound method NDFrame.describe of                                                                                                                                                        url      make_model                                   short_description body_type  price             vat     km registration prev_owner  kW      hp                 Type Previous Owners Next Inspection Inspection new Warranty Full Service Non-smoking Vehicle null         Make             Model Offer Number First Registration      Body Color      Paint Type Body Color Original   Upholstery           Body Nr. of Doors Nr. of Seats Model Code         Gearing Type    Displacement Cylinders          Weight Drive chain                Fuel                                                               Consumption CO2 Emission Emission Class                                                                                                                                                                                

In [88]:
oto["Gears"].value_counts()

[\n6\n]     5822
[\n5\n]     3239
[\n7\n]     1908
[\n8\n]      224
[\n9\n]        6
[\n4\n]        2
[\n3\n]        2
[\n1\n]        2
[\n50\n]       1
[\n2\n]        1
Name: Gears, dtype: int64